In [394]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [395]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 15
EVENT_NAME = 'CUSUM_0.002'
LABEL_NAME = "TB"

START_DATE = "20210101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "interm/events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [396]:
DIRECTION_LABEL_DIR = BASE_DIR / "labels/direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [397]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 1.23 ms, sys: 13 ms, total: 14.2 ms
Wall time: 13.8 ms


In [398]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,minute_sin,minute_cos,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-18 02:30:00,103.7450,103.7455,103.6915,103.7075,1.954470e+12,0.002984,-0.000352,-0.0365,-0.000352,1.871269,...,5.665539e-16,-1.000000e+00,0.500000,0.866025,0.000000,1.000000,-0.485302,-0.874347,0.5,0.866025
2021-01-19 02:00:00,103.8350,103.8945,103.8295,103.8800,4.718710e+12,0.002695,0.000433,0.0450,0.000433,2.014353,...,0.000000e+00,1.000000e+00,0.500000,0.866025,0.781831,0.623490,-0.651372,-0.758758,0.5,0.866025
2021-01-20 00:15:00,103.8715,103.8935,103.8420,103.8445,1.830570e+12,0.003199,-0.000265,-0.0275,-0.000265,1.886134,...,1.000000e+00,2.832769e-16,0.000000,1.000000,0.974928,-0.222521,-0.790776,-0.612106,0.5,0.866025
2021-01-20 15:00:00,103.7785,103.7785,103.5575,103.5805,4.527180e+12,0.001655,-0.001893,-0.1965,-0.001895,1.725708,...,0.000000e+00,1.000000e+00,-0.707107,-0.707107,0.974928,-0.222521,-0.790776,-0.612106,0.5,0.866025
2021-01-21 04:45:00,103.4580,103.4610,103.3905,103.4005,2.107770e+12,0.002662,-0.000546,-0.0565,-0.000546,1.812448,...,-1.000000e+00,-1.836970e-16,0.866025,0.500000,0.433884,-0.900969,-0.897805,-0.440394,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL


In [399]:
import pandas_ta as ta

In [400]:
# 預計持單 1 小時 (60 minutes)
# 計算 1 小時內價格可能到哪裡
# 計算 1 小時 normalized atr
atr12 = df.ta.natr(length=12)

In [401]:
atr12

timestamp
2021-01-15 14:15:00         NaN
2021-01-15 14:30:00         NaN
2021-01-15 14:45:00         NaN
2021-01-15 15:00:00         NaN
2021-01-15 15:15:00         NaN
                         ...   
2024-12-30 22:45:00    0.048840
2024-12-30 23:00:00    0.053224
2024-12-30 23:15:00    0.051785
2024-12-30 23:30:00    0.052297
2024-12-30 23:45:00    0.051264
Name: NATR_12, Length: 98795, dtype: float64

In [402]:
trgt = atr12.reindex(t_events, method='ffill') / 100 # rate

In [403]:
(trgt).describe()

count    6658.000000
mean        0.001084
std         0.000626
min         0.000164
25%         0.000680
50%         0.000943
75%         0.001297
max         0.005998
Name: NATR_12, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [404]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns (rate)
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [405]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [406]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [407]:
t1

timestamp
2021-01-18 02:30:00   2021-01-18 04:30:00
2021-01-19 02:00:00   2021-01-19 04:00:00
2021-01-20 00:15:00   2021-01-20 02:15:00
2021-01-20 15:00:00   2021-01-20 17:00:00
2021-01-21 04:45:00   2021-01-21 06:45:00
                              ...        
2024-12-30 11:15:00   2024-12-30 13:15:00
2024-12-30 14:30:00   2024-12-30 16:30:00
2024-12-30 15:00:00   2024-12-30 17:00:00
2024-12-30 15:30:00   2024-12-30 17:30:00
2024-12-30 20:30:00   2024-12-30 22:30:00
Name: timestamp, Length: 6658, dtype: datetime64[ns]

In [408]:
trgt.describe()

count    6658.000000
mean        0.001084
std         0.000626
min         0.000164
25%         0.000680
50%         0.000943
75%         0.001297
max         0.005998
Name: NATR_12, dtype: float64

In [409]:
minTrgt = trgt.quantile(.3)
minTrgt
# minTrgt = 0

np.float64(0.0007349115487366089)

In [410]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 1.06 s, sys: 17.4 ms, total: 1.08 s
Wall time: 1.07 s


In [411]:
events.head()

,t1,trgt
timestamp,,
2021-02-05 14:30:00,2021-02-05 16:30:00,0.000908
2021-02-17 14:30:00,2021-02-17 16:30:00,0.000808
2021-02-22 14:45:00,2021-02-22 15:00:00,0.000809
2021-02-25 18:15:00,2021-02-25 19:45:00,0.000869
2021-02-25 20:15:00,2021-02-25 22:15:00,0.000861


In [412]:
events = events.dropna().copy()

# Keep only intraday events

In [413]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [414]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [415]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [416]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    4557.000000
mean        0.001611
std         0.001326
min         0.000000
25%         0.000962
50%         0.001371
75%         0.001949
max         0.028329
Name: ret, dtype: float64
0.001047317100665457


In [417]:
labels.bin.value_counts()

bin
 0.0    1563
 1.0    1522
-1.0    1472
Name: count, dtype: int64

In [418]:
labels["bin_class"] = labels["bin"] + 1

In [419]:
labeled_events = events.join(labels, how='inner')

In [420]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2021-02-05 14:30:00,2021-02-05 16:30:00,0.000908,-0.000659,0.0,1.0
2021-02-17 14:30:00,2021-02-17 16:30:00,0.000808,-0.000666,0.0,1.0
2021-02-22 14:45:00,2021-02-22 15:00:00,0.000809,-0.001046,-1.0,0.0
2021-02-25 18:15:00,2021-02-25 19:45:00,0.000869,0.001018,1.0,2.0
2021-02-25 20:15:00,2021-02-25 22:15:00,0.000861,0.000047,0.0,1.0


## Save events to disk

In [421]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [422]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/labels/direction_labels/USDJPY-15m-20210101-20241231-CUSUM_0.002-TB.pkl')